In [1]:
import os
os.chdir("..")
from parc import model
from parc import IO
from parc import losses
import keras

#initialize input data characteristics
input_img_size = 485  #image size
case_numbers = 1     #number of cases
time_steps = 20       #number of time steps
del_t = 0.79*(10**-9) #time step size

#initialize adaptable model parameters
input_size = (None,None,2)
depth = 3
kernel_size = 5
numFeatureMaps = 128

#Parse the raw data and return microstructure data and temperature/pressure outputs
data_in, total_output = IO.parseData("data/raw",input_img_size,case_numbers,time_steps,del_t)

#Split data into training, validation, and testing sets
X_train, y_train, X_val, y_val, test_X, test_Y = IO.splitData(data_in,total_output,splits=[1, 0, 0])

#reshape to match old data format
y_train = IO.reshape(y_train)
y_val = IO.reshape(y_val)
test_Y = IO.reshape(test_Y)

#initilize model and model parameters
parc_model = model.parc(input_size,time_steps,depth,kernel_size,numFeatureMaps)
loss = ["mse", "mse"]
loss_weights = [1, 1]
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
metrics = ["mae"]
parc_model.compile(loss = loss,loss_weights = loss_weights,optimizer = optimizer,metrics = metrics)

x_tn_init = y_train[:,:,:,0,:1]
x_val_init = y_val[:,:,:,0,:1]

#train the model using training set 
history = parc_model.fit(
    x=[X_train[:,:,:,:], x_tn_init],
    y=[y_train[:, :, :, :, :1], y_train[:, :, :, :, 2:3]],
    validation_data=([X_val[:,:,:,:], x_val_init], [y_val[:, :, :, :, :1], y_val[:, :, :, :, 2:3]]),
    batch_size=1,
    epochs=2,
)

#prediction using trained model parameters
pred = parc_model.predict([test_X,test_Y[:,:,:,0,:1]])

#calculate losses
rmse = losses.rmse(test_Y,pred[0][:,:,:,:,:])
r2 = losses.r2(test_Y,pred[0][:,:,:,:,:])
loss_cv = losses.step_weighted_loss(test_Y,pred[0][:,:,:,:,:])
step_loss = losses.step_weighted_physical_loss(test_Y,pred[0][:,:,:,:,:])
state_loss = losses.state_weighted_loss(test_Y,pred[0][:,:,:,:,:])

#create plots for data
graphs.plot_rmse()

max and min of channel 0 are:  4000.0 300.0
max and min of channel 1 are:  32479223000.0 -1310510500.0
max and min of channel 2 are:  4502356354430.379 -1994580151898.7341
max and min of channel 3 are:  4.111294050632911e+19 -3.252435227848101e+19
Finished Processing Data
shape of microstructure data is:  (1, 240, 240, 2)
shape of output data is:  (1, 240, 240, 20, 4)
1
1
1
1
(1, 240, 240, 2)
(1, 240, 240, 20, 4)
(0, 240, 240, 2)
(0, 240, 240, 20, 4)
(0, 240, 240, 2)
(0, 240, 240, 20, 4)
Old shape of data:  (1, 240, 240, 20, 4)
New shape of data:  (1, 240, 240, 78)
Old shape of data:  (0, 240, 240, 20, 4)
New shape of data:  (0, 240, 240, 78)
Old shape of data:  (0, 240, 240, 20, 4)
New shape of data:  (0, 240, 240, 78)


IndexError: too many indices for array: array is 4-dimensional, but 5 were indexed